## Programação Paralela
***

**Concorrência**: é quando um computador que possui apenas um core parece estar realizando duas ou mais operações ao mesmo tempo, quando na verdade está alternando a execução destas operações de forma tão rápida que temos a ilusão de que tudo é executado simultaneamente.

**Paralelismo**: é quando um computador que possui dois ou mais cores executa operações realmente de forma paralela, utilizando para isso os cores disponíveis, ou seja, se um determinado computador tem 2 cores, posso ter duas operações sendo executadas paralelamente cada uma em um core diferente.

Infelizmente o GIL (Global Interpreter Lock do Python) é restritivo quanto ao uso de threads paralelas em Python, porém o módulo **concurrent.futures** permite que possamos utilizar múltiplos cores. Para isso, este módulo "engana" o GIL criando novos interpretadores como subprocessos do interpretador principal. Desta maneira, cada subprocesso tem seu próprio GIL e, por fim, cada subprocesso tem um ligação com o processo principal, de forma que recebem instruções para realizar operações e retornar resultados.

***
### Métodos Importantes
***

O módulo [concurrent.futures](https://docs.python.org/3.6/library/concurrent.futures.html#processpoolexecutor) do python3 fornece uma interface de alto nível para a execução assíncrona de chamadas.

A execução assíncrona pode ser executada com encadeamentos, usando **ThreadPoolExecutor** ou processos separados, usando **ProcessPoolExecutor**. Ambos implementam a mesma interface, definida pela classe abstrata **Executor**.

Seus métodos principais são:

* **.submit(função, vetor_de_argumentos, dicionario_de_argumentos)**: Executa a função passando seus argumentos e retorna um objeto **Future** representando a execução da função.


* **.map(função, iteravel, timeout, chunksize)**: Várias chamadas assíncronas da função são feitas simultaneamente para cada item da lista (iteravel). Pode-se inserir um tempo limite de espera (timeout) que se passar dispara uma exceção **TimeoutError**

> Ao usar ProcessPoolExecutor, esse último parâmetro corta iteráveis em um número de partes que ele envia ao pool como tarefas separadas. O tamanho (aproximado) desses blocos pode ser especificado configurando **chunksize** para um inteiro positivo. Para iteraveis muito longo, usando um valor grande para chunksize pode melhorar significativamente o desempenho em comparação com o tamanho padrão de 1. Com ThreadPoolExecutor, chunksize não tem efeito.

* **.shutdown(wait=True)**: Sinalize ao executor que ele deve liberar quaisquer recursos que esteja usando quando os futuros atualmente pendentes forem executados.

> Se wait for True, este método não retornará até que todos os futuros pendentes sejam executados e os recursos associados ao executor tenham sido liberados. Se wait for False então este método retornará imediatamente e os recursos associados ao executor serão liberados quando todos os futuros pendentes forem executados. Independentemente do valor de espera, todo o programa Python não será encerrado até que todos os futuros pendentes sejam executados.

> Você pode evitar ter que chamar esse método explicitamente se usar a instrução **with**, que desligará o Executor (esperando como se Executor.shutdown() fosse chamado com wait configurado como True).

* **.wait(função, timeout, return_when=FIRST_COMPLETED)**: Cria um tupla de funções (futures) que já foram executadas (done) e que ainda estão na lista (not_done).

***
### Exemplos
***

Agora que já vimos um pouco de teoria vamos colocar em prática o uso do concurrent.futures. Vamos supor que tenhamos um lista de preços e que queremos aumentar em 10% o valor de cada item.

***

In [1]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from datetime import datetime
import time

In [2]:
def generate_list():
    """
    Gera a lista de preços
    """

    result = []
    for i in range(0, 20):
        result.append(pow(i, 2) * 42)

    return result

In [3]:
def increase_price_by_10_percent(price):
    """
    Faz um cálculo maroto para que seja realizado
    não tão rapido.
    """
    
    print("Execução do preço: " + str(price))
    time.sleep(1)

    price += price / 10 * 100

    return price

In [4]:
def increase_price_serial(price_list, increase_function):
    """
    Calcula a lista de preços acrescido de 10% de forma serial.

    Aqui a função passada pelo parâmetro increase_function será executada para
    cada item da price_list de forma sequencial.
    """
    
    start = datetime.now()

    result = list(map(increase_function, price_list))

    end = datetime.now()

    print("Tomou {} segundos para incrementar os valores dos preços".format((end - start).total_seconds()))

In [5]:
def increase_price_with_threads(price_list, increase_function):
    """
    Calcula a lista de preços acrescido de 10% usando threads.
    
    Aqui já começamos a fazer uso da classe ThreadPoolExecutor, e que vai nos permitir executar
    a increase_function de forma concorrente. Note que ao instanciar ThreadPoolExecutor estamos
    passando o parâmetro max_workers=2, isto está indicando o numero máximo de threads que será
    usado para executar as operações.
    """

    start = datetime.now()

    pool = ThreadPoolExecutor(max_workers=4)
    results = list(pool.map(increase_function, price_list))
    pool.shutdown()

    end = datetime.now()

    print("Tomou {} segundos para incrementar os valores dos preços com threads".format((end - start).total_seconds()))

In [6]:
def increase_price_with_subprocess(price_list, increase_function):
    """
    Calcula a lista de preços acrescido de 10% usando subprocessos.
    
    Nesta função estamos fazendo uso da classe ProcessPoolExecutor que tem a funionalidade
    bastante semelhante à classe ThreadPoolExecutor exceto pelo fato de que esta classe
    permite que a função increase_function() seja executada realmente de forma paralela.
    
    Essa "mágica" é conseguida da seguinte forma:
    
    - Cada item da lista de preços é serializado através do pickle
    
    - Os dados serializados são copiados do processo principal para os processos filhos
    por meio de um socket local.
    
    - Aqui o pickle entra em cena novamente para deserializar os dados para os subprocessos;
    
    - Os subprocessos importam o módulo Python que contém a função que será utilizada; no nosso
    caso, será importado o módulo onde increase_function está localizada.
    
    - As funções são executadas de forma paralela em cada subprocesso
    
    - O resultado destas funções é serializado e copiado de volta para o processo principal via socket;
    
    - Os resultados são desserializados e mesclados em uma lista para que possam ser retornados
    """

    start = datetime.now()

    pool = ProcessPoolExecutor(max_workers=4)
    results = list(pool.map(increase_function, price_list))
    pool.shutdown()

    end = datetime.now()

    print("Tomou {} segundos para incrementar os valores dos preços com subprocessos".format((end - start).total_seconds()))

In [7]:
prices = generate_list()
print(prices)

[0, 42, 168, 378, 672, 1050, 1512, 2058, 2688, 3402, 4200, 5082, 6048, 7098, 8232, 9450, 10752, 12138, 13608, 15162]


In [8]:
increase_price_serial(prices, increase_price_by_10_percent)

Execução do preço: 0
Execução do preço: 42
Execução do preço: 168
Execução do preço: 378
Execução do preço: 672
Execução do preço: 1050
Execução do preço: 1512
Execução do preço: 2058
Execução do preço: 2688
Execução do preço: 3402
Execução do preço: 4200
Execução do preço: 5082
Execução do preço: 6048
Execução do preço: 7098
Execução do preço: 8232
Execução do preço: 9450
Execução do preço: 10752
Execução do preço: 12138
Execução do preço: 13608
Execução do preço: 15162
Tomou 20.034298 segundos para incrementar os valores dos preços


In [9]:
increase_price_with_threads(prices, increase_price_by_10_percent)

Execução do preço: 0
Execução do preço: 42
Execução do preço: 168
Execução do preço: 378
Execução do preço: 672Execução do preço: 1050

Execução do preço: 1512
Execução do preço: 2058
Execução do preço: 2688
Execução do preço: 3402
Execução do preço: 4200
Execução do preço: 5082
Execução do preço: 6048
Execução do preço: 7098Execução do preço: 8232

Execução do preço: 9450
Execução do preço: 10752
Execução do preço: 12138
Execução do preço: 13608
Execução do preço: 15162
Tomou 5.023675 segundos para incrementar os valores dos preços com threads


In [10]:
increase_price_with_subprocess(prices, increase_price_by_10_percent)

Execução do preço: 0
Execução do preço: 42
Execução do preço: 168
Execução do preço: 378
Execução do preço: 672
Execução do preço: 1050
Execução do preço: 1512
Execução do preço: 2058
Execução do preço: 2688
Execução do preço: 3402
Execução do preço: 5082
Execução do preço: 4200
Execução do preço: 6048
Execução do preço: 7098
Execução do preço: 8232
Execução do preço: 9450
Execução do preço: 12138
Execução do preço: 13608
Execução do preço: 10752
Execução do preço: 15162
Tomou 5.08382 segundos para incrementar os valores dos preços com subprocessos


Nota-se que a classe ProcessPoolExecutor faz muitos "malabarismos" para que o paralelismo seja realmente possível. Por isso só deve ser usadas sé realmente o método exigir o paralelismo, em métodos simples ele se torna mais lento que threads e sequêncial.

***
### Outro Exemplo
***

In [11]:
def increase_price_by_10_percent(price):
    """
    Faz um cálculo maroto para que seja realizado
    não tão rapido.
    """
    
    print("Cálculo do preço: " + str(price))
    time.sleep(5)

    price += price / 10 * 100
    print("Novo preço: " + str(price))

    return price

In [12]:
from concurrent.futures import wait, FIRST_COMPLETED, thread

prices = [125, 250, 99, 23]

with ThreadPoolExecutor(max_workers=len(prices)) as executor:

    futures = [executor.submit(increase_price_by_10_percent, prices[i]) for i in range(len(prices))]

    done, not_done = wait(futures, return_when=FIRST_COMPLETED)

    for completed in done:
        print("RETORNOU")
        print(completed.result())
        executor._threads.clear()
        thread._threads_queues.clear()
        executor.shutdown(wait=False)

Cálculo do preço: 125Cálculo do preço: 250

Cálculo do preço: 99
Cálculo do preço: 23
Novo preço: 252.99999999999997
RETORNOU
Novo preço: 2750.0Novo preço: 1375.0
252.99999999999997
Novo preço: 1089.0

